In [7]:
import pandas as pd
import pandas_datareader as pdr
import numpy as np

In [8]:
ff5_month = pdr.get_data_famafrench('F-F_Research_Data_5_Factors_2x3', start='1963-07', end='2023-03')[0]
ff5_month = ff5_month / 100
mom_month = pdr.get_data_famafrench('F-F_Momentum_Factor', start='1927-01', end='2023-03')[0]
mom_month = mom_month / 100

ff5_month = ff5_month.reset_index()
mom_month = mom_month.reset_index()
ff5mom_month = ff5_month.merge(mom_month, on='Date', how='right')
ff5mom_month = ff5mom_month.set_index('Date')

In [9]:
ff5_daily = pdr.get_data_famafrench('F-F_Research_Data_5_Factors_2x3_daily', start='1963-07-01', end='2023-03-31')[0]
ff5_daily = ff5_daily / 100
mom_daily = pdr.get_data_famafrench('F-F_Momentum_Factor_daily', start='1926-11-03', end='2023-03-31')[0]
mom_daily = mom_daily / 100

ff5_daily = ff5_daily.reset_index()
mom_daily = mom_daily.reset_index()
ff5mom_daily = ff5_daily.merge(mom_daily, on='Date', how='right')
ff5mom_daily = ff5mom_daily.set_index('Date')

In [10]:
ff5mom_month

,Mkt-RF,SMB,HML,RMW,CMA,RF,Mom
Date,,,,,,,
1927-01,NaN,NaN,NaN,NaN,NaN,NaN,0.0036
1927-02,NaN,NaN,NaN,NaN,NaN,NaN,-0.0214
1927-03,NaN,NaN,NaN,NaN,NaN,NaN,0.0361
1927-04,NaN,NaN,NaN,NaN,NaN,NaN,0.0430
1927-05,NaN,NaN,NaN,NaN,NaN,NaN,0.0300
...,...,...,...,...,...,...,...
2022-11,0.0460,-0.0267,0.0138,0.0601,0.0311,0.0029,-0.0201
2022-12,-0.0641,-0.0016,0.0132,0.0009,0.0419,0.0033,0.0452
2023-01,0.0665,0.0443,-0.0405,-0.0262,-0.0453,0.0035,-0.1598


In [11]:
ff5mom_daily

,Mkt-RF,SMB,HML,RMW,CMA,RF,Mom
Date,,,,,,,
1926-11-03,NaN,NaN,NaN,NaN,NaN,NaN,0.0056
1926-11-04,NaN,NaN,NaN,NaN,NaN,NaN,-0.0050
1926-11-05,NaN,NaN,NaN,NaN,NaN,NaN,0.0117
1926-11-06,NaN,NaN,NaN,NaN,NaN,NaN,-0.0003
1926-11-08,NaN,NaN,NaN,NaN,NaN,NaN,-0.0001
...,...,...,...,...,...,...,...
2023-03-27,0.0027,0.0051,0.0102,-0.0028,0.0035,0.00016,0.0082
2023-03-28,-0.0017,-0.0003,0.0074,0.0008,0.0055,0.00016,0.0073
2023-03-29,0.0139,-0.0034,-0.0050,-0.0090,-0.0054,0.00016,-0.0111


In [12]:
# calculate rolling variance
rolling_window = 12 # You can adjust this value
ff5mom_month_rolling_var = ff5mom_month.rolling(window=rolling_window).var()

# reshape the monthly factor dataframe
ff5mom_month_rolling_var = ff5mom_month_rolling_var.reset_index().melt(id_vars='Date', var_name='Factor', value_name='RollingVar')
ff5mom_month_reshaped = ff5mom_month.reset_index().melt(id_vars='Date', var_name='Factor', value_name='FactorReturns')

# merge the reshaped monthly factor dataframe with the rolling variance dataframe
df = pd.merge(ff5mom_month_reshaped, ff5mom_month_rolling_var, on=['Date', 'Factor'])

# calculate inverse of rolling variance
df['InvRollingVar'] = 1 / df['RollingVar']

# group by date to calculate weights
df['Weight'] = df.groupby('Date')['InvRollingVar'].apply(lambda x: x / x.sum())

# calculate lag weights using shift
df['LagWeight'] = df.groupby('Factor')['Weight'].shift()

# calculate monthly volatility-managed portfolio excess returns
df['ExcessReturns'] = df['LagWeight'] * df['FactorReturns']
monthly_excess_returns = df.groupby('Date')['ExcessReturns'].sum()


In [18]:
def estimate_models(excess_returns, ff5):
    market_excess = ff5['Mkt-RF']
    portfolio_excess = excess_returns
    market_excess = market_excess.loc[portfolio_excess.index]

    print(portfolio_excess[portfolio_excess.isna()])

    # CAPM Model
    X = add_constant(market_excess)
    capm_model = OLS(portfolio_excess, X).fit()

    # FF3 Model
    X = add_constant(ff5[['Mkt-RF', 'SMB', 'HML']].loc[portfolio_excess.index])
    ff3_model = OLS(portfolio_excess, X).fit()

    # Carhart Model
    X = add_constant(ff5.loc[portfolio_excess.index].join(ff5['RMW']).join(ff5['CMA']))
    carhart_model = OLS(portfolio_excess, X).fit()

    # FF5 Model
    X = add_constant(ff5.loc[portfolio_excess.index])
    ff5_model = OLS(portfolio_excess, X).fit()

    return capm_model, ff3_model, carhart_model, ff5_model

In [19]:
capm_model, ff3_model, carhart_model, ff5_model = estimate_models(monthly_excess_returns, ff5_month)

KeyError: "None of [PeriodIndex(['1927-01', '1927-02', '1927-03', '1927-04', '1927-05', '1927-06',\n             '1927-07', '1927-08', '1927-09', '1927-10',\n             ...\n             '2022-06', '2022-07', '2022-08', '2022-09', '2022-10', '2022-11',\n             '2022-12', '2023-01', '2023-02', '2023-03'],\n            dtype='period[M]', name='Date', length=1155)] are in the [index]"

In [ ]:
import matplotlib.pyplot as plt

# calculate log cumulative returns
portfolio_cum_returns = np.log(1 + monthly_excess_returns).cumsum()
market_cum_returns = np.log(1 + market_returns).cumsum()

# plot
plt.figure(figsize=(12, 8))
plt.plot(portfolio_cum_returns, label='Volatility-Managed Portfolio')
plt.plot(market_cum_returns, label='Market')
plt.xlabel('Date')
plt.ylabel('Log Cumulative Returns')
plt.title('Log Cumulative Returns')
plt.legend()
plt.show()